# Описание задания

В рамках данного задания, студент должен создать и оценить 4 типа рекомендательных систем:
* Content-based RS

Каждая рекомендательная система - отдельное подзадание. Подзадание считается выполненным, если студент создал рекомендательную систему, которая **лучше (или хуже, но не более чем на 10%)** системы, созданной автором данного блокнота. Системы оцениваются с использованием метрики ``HR@N``, описанной ниже.

# Детальное описание

#### Данные: 
Датасет представлен множеством отзывов к компьютерным играм (объектам) от пользователей Amazon. Каждый отзыв представлен в виде JSON-структуры со следующими полями:
* идентификатор пользователя - reviewerID
* идентификатор объекта - asin
* текст отзыва - reviewText
* рейтинг - overall
* время публикации обзора - unixReviewTime
* другие поля, не использованные автором этого блокнота (смотри полное описание JSON [тут](http://jmcauley.ucsd.edu/data/amazon/))

У каждого объекта есть как минимум 5 отзывов, каждый пользователь написал как минимум 5 отзывов. 
#### Цель: 
Построить рекомендательную систему, предсказывающую объекты, которые пользователь приобретет в ближайшем будущем. Для упрощения мы считаем, что пользователь приобрел объект, если он написал про него отзыв.
#### Подготовка данных:
Данные разделены на тренировочную и тестовую выборки по времени публикации отзывов. Первые 80% данных (более старые) используются как тренировочная выборка, остальные - как тестовая. 

Построение рекомендательной системы (т.е., выбор и тренировка моделей, оптимизация параметров и т.д.) осуществляется **только** с использованием тренировочной выборки. Все параметры, использованные в моделях, **должны быть** получены или объяснены с помощью тренировочных данных. Студент вправе использовать тренировочную выборку как его душе угодно. 

Тестирующая выборка используется **только** для оценки рекомендательной системы.

Для построения рекомендательных моделей также можно использовать JSON-поля из датасета, неиспользованные автором этого блокнота.
#### Оценка качества рекомендательной системы
Цель рекомендательной системы - посоветовать пользователю объекты, которые он захочет приобрести. Для оценки качества такой системы мы воспользуемся метрикой `hit-ratio (HR)`. 

$$
HR = \frac{1}{|U_T|}\sum_{u \in U_T} \mathrm{I}(Rel_u \cap Rec_u)
$$

* $U_T$ - множество пользователей из тестовой выборки
* $Rec_u$ - множество объектов, рекомендованных пользователю $u$ 
* $Rel_u$ - множество объектов, оцененных пользователем $u$ в тестовой выборке
* $\mathrm{I}(Rel_u \cap Rec_u)$ - бинарная функция-индикатор. Функция возвращает 1 если $Rel_u \cap Rec_u \ne \emptyset$, иначе 0

$HR=1$ если для каждого пользователя мы рекомендовали хотя бы один релевантный объект. Так как обычно пользователи просматривают только первые $N$ рекомендаций, мы будем считать метрику $HR@N$, где $N=10$ (т.е. множество $Rec_u$ будет содержать только 10 объектов). 

# Условные обозначения
* `uid` - идентификатор пользователя
* `iid` - идентификатор объекта

# Games RSs

In [15]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [16]:
JSON_DATA_PATH = "data/Video_Games_5.json"

## Анализ данных

In [17]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [18]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [19]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [20]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [21]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [22]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "data/Video_Games_5.json"
N = 10

In [23]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [24]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

Для упрощения тестирования предлагается использовать словарь следующего типа:

```python
recs = {
    uid_1: {
        iid_1: score_11,
        iid_2: score_12,
        ...
    },
    uid_2: {
        iid_1: score_21,
        iid_2: score_22,
        ...
    },
    ...
}
```

где `uid_i` - идентификатор тестового пользователя, `iid_j` - идентификатор рекомендованного объекта, а `score_ij` - предсказанный рейтинг/вес объекта `j` для пользователя `i`.

In [25]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [26]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Content-based RS

Простая content-based рекомендательная система описывает пользователей и объекты как вектора в некотором N-мерном пространстве фич. Вектор объекта показывает, насколько объект принадлежит к той или иной фиче. Вектор пользователя показывает, насколько пользователь предпочитает ту или иную фичу. Рекомендации строятся путем поиска объектов, чьи вектора похожи на вектор предпочтений пользователя. Предполагается, что чем более похожи вектора пользователя и объекта, тем интереснее этот объект пользователю.

#### `HR@10` для content-based, модели созданной автором блокнота: 0.065

### Сделаем предобработку rewiew

In [27]:
# соберем данные для обучения TfidfVectorizer
texts = [m for m in training_df['review']]
len(texts)

185427

In [28]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', analyzer='word', tokenizer=None, max_features=25000 )
#tfidf = TfidfVectorizer(stop_words='english', analyzer='word', tokenizer=None )
tfidf.fit(texts)

Wall time: 42.5 s


In [29]:
result = sorted(tfidf.vocabulary_.items(), key=lambda x:x[1], reverse=True)[:20]
result

[('zx', 24999),
 ('zwei', 24998),
 ('zurg', 24997),
 ('zune', 24996),
 ('zumba', 24995),
 ('zuma', 24994),
 ('zr', 24993),
 ('zork', 24992),
 ('zora', 24991),
 ('zoos', 24990),
 ('zooms', 24989),
 ('zooming', 24988),
 ('zoomed', 24987),
 ('zoom', 24986),
 ('zoo', 24985),
 ('zoning', 24984),
 ('zoni', 24983),
 ('zones', 24982),
 ('zone', 24981),
 ('zonda', 24980)]

## Посчитаем TFIDF

In [30]:
# сделаем матрицу из обьектов и их описаний
training_df['review']='  '+training_df['review']
#Object=training_df.groupby('iid')['review'].sum()
#len(Object)

In [31]:
# попробуем как Турал
review_ftr_m = tfidf.transform(training_df.review)
#сразу умножаем на рейтинги
review_ftr_m = review_ftr_m.multiply(training_df.rating.values.reshape(-1, 1)).tocsr()

In [32]:
# маритцу получили, по хорошему ее бы не мешало почистить и обработать
# как это делать пока не очень понятно
from sklearn.preprocessing import normalize
from scipy.sparse import vstack

# возьмем функцию Турала, она быстрее
def _prepare_iid_data(df, review_ftr_m): 
    iid_to_row = {}
    rows = []
    # не самый оптимальный group by       
    for row_id, iid in enumerate(df.iid.unique()):
        iid_to_row[iid] = row_id # свяжем
        
        # вариант ищем iddtf * рейтинг и потом складываем вектора
        iid_ftr_m = csr_matrix(review_ftr_m[np.where(df.iid == iid)[0]].sum(axis=0))
        rows.append(iid_ftr_m)
    
    iid_ftr_m = normalize(vstack(rows, format='csr'))
    return iid_to_row, iid_ftr_m


In [33]:
# маритцу получили, по хорошему ее бы не мешало почистить и обработать
# как это делать пока не очень понятно
from sklearn.preprocessing import normalize
from scipy.sparse import vstack

In [34]:
%%time
iid_to_row, iid_profile = _prepare_iid_data(training_df, review_ftr_m)
len(iid_to_row)

Wall time: 19.3 s


In [35]:
# получили большую матрицу обьектов
iid_profile

<10098x25000 sparse matrix of type '<class 'numpy.float64'>'
	with 6962533 stored elements in Compressed Sparse Row format>

## Создадим профиль пользователя

### Поработаем с рейтингом пользователя

In [36]:
# сделаем пока просто подсчитаем средний рейтинг который пользователь ставит и если текущий быше нравится
# можно примеить специальные функции для рейтига /// если хватит времени
#mean_tr=training_df.groupby('uid')['rating'].mean().to_frame()
#training_df = pd.merge(training_df, mean_tr, how='left', left_on='uid', right_index=True )

In [37]:
# народ настроен  более положительно чем отрицательно, это хорошо
# точно нужно применять другую функцию для оценки рейтинга и возможно поробовать сделать нравится/не нравится/нормально
#training_df["liked"] = (training_df.rating_x >= training_df.rating_y).astype(int)
#training_df.liked.value_counts()

In [38]:
training_df.rename(columns={'rating_x' : 'rating'}, inplace=True)
#training_df.columns.values[3] = 'rating'
training_df.query('uid==5')

,uid,iid,review,rating,dt
5,5,0,"Overall this is a well done racing game, wit...",4.0,1368230400
93459,5,5000,My family loves this game. We all play toge...,5.0,1301788800
121923,5,6308,Need for Speed - Shift is a great racing gam...,4.0,1346630400
123196,5,6349,Overall this is a well done racing racing ga...,4.0,1368230400
125474,5,6428,I have read the many negative reviews. I pe...,5.0,1264809600
168532,5,8069,I find Need for Speed Shift 2 a very fun gam...,5.0,1309824000
214274,5,9935,Graphics are excellent. Would be nice to be...,5.0,1360454400


In [39]:
temp_crs=csr_matrix(tfidf.transform(training_df[training_df.uid == 5].review)).transpose()
temp_crs=temp_crs.multiply(training_df[training_df.uid == 5].rating)
temp_crs=csr_matrix(temp_crs.sum(axis=1))

temp_crs

#*training_df[training_df.uid == 5].rating

<25000x1 sparse matrix of type '<class 'numpy.float64'>'
	with 213 stored elements in Compressed Sparse Row format>

In [40]:
# Моя версия
def _prepare_uid_data2(df, tfidf):  
        uid_to_row = {}
        rows = []
        
        # вытаскиваем все существующие отзывы поьзователя
        for row_id, uid in enumerate(df.uid.unique()):
            uid_to_row[uid] = row_id # свяжем
            
            # вариант ищем iddtf * рейтинг и потом складываем вектора
            uid_ftr_m=csr_matrix(tfidf.transform(training_df[training_df.uid == uid].review))#.transpose()
            #uid_ftr_m=uid_ftr_m.multiply(training_df[training_df.uid == uid].rating).transpose()
            uid_ftr_m=csr_matrix(uid_ftr_m.sum(axis=0))
            
            rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m
            
            

In [41]:
%%time
# постоим матрицу пользователей
#uid_to_row, uid_profile = _prepare_uid_data2(training_df, tfidf)
#len(uid_to_row)

Wall time: 0 ns


In [42]:
# версия Турала
def _prepare_uid_data(df, iid_to_row, iid_ftr_m):  
        uid_to_row = {}
        rows = []
        
        # gr_df - кусок df с данными одного пользователя 
        for gr_id, gr_df in df.groupby("uid"):
            uid = gr_df.uid.values[0]
            
            # поиск объектов и пользовательских рейтингов для них
            iid_rows = []
            ratings = []
            for iid, rating in zip(gr_df.iid.values, gr_df.rating.values):
                if iid in iid_to_row:
                    iid_rows.append(iid_to_row[iid])
                    ratings.append(rating)
                  
            # создание профиля пользователя
            if iid_rows:
                ratings = np.array(ratings).reshape(-1, 1)
                uid_ftr_m = csr_matrix(iid_ftr_m[iid_rows].multiply(ratings).sum(axis=0)) 
                uid_to_row[uid] = len(uid_to_row)
                rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m

In [43]:
%%time
uid_to_row, uid_profile = _prepare_uid_data(training_df, iid_to_row, iid_profile)

Wall time: 1min 4s


In [44]:
uid_profile

<22215x25000 sparse matrix of type '<class 'numpy.float64'>'
	with 126610389 stored elements in Compressed Sparse Row format>

In [45]:
# немного упрощаем себе жизнь (для подсчета рекомендаций с использованием косинусной меры)
ftr_iid_m = iid_profile.T.tocsr()
row_to_iid = {row_id: iid for iid, row_id in iid_to_row.items()}

In [50]:
u_row_id = uid_to_row[256]
print(u_row_id)
u_row = uid_profile[u_row_id]
print(u_row)
u_recs = u_row.dot(ftr_iid_m)
#print(u_recs)
#np.sort(u_recs.data, axis=None)[-10:]

234
  (0, 0)	0.00174386680255
  (0, 1)	0.00277064883165
  (0, 4)	0.000622988154355
  (0, 7)	0.00025406575839
  (0, 20)	0.0202589782985
  (0, 21)	0.00644542842729
  (0, 22)	0.00174945301537
  (0, 80)	0.00244357577275
  (0, 81)	0.00490923154001
  (0, 84)	0.00149185436162
  (0, 85)	0.0144799426578
  (0, 99)	0.00313623542357
  (0, 102)	0.000735633594859
  (0, 107)	0.00465106200227
  (0, 114)	0.00229190619809
  (0, 131)	0.000185439528416
  (0, 135)	0.00253744732123
  (0, 175)	0.00985822850235
  (0, 177)	0.0101798048605
  (0, 178)	0.0106121492067
  (0, 180)	0.000505648872891
  (0, 182)	0.00166753010904
  (0, 183)	0.00357905465136
  (0, 184)	0.0106043874292
  (0, 185)	0.0071149595237
  :	:
  (0, 24796)	0.000706318499395
  (0, 24812)	0.00531869009452
  (0, 24824)	0.0161235081302
  (0, 24843)	0.00562264487896
  (0, 24848)	0.000798186794292
  (0, 24850)	0.00401182032974
  (0, 24851)	0.0302572948692
  (0, 24855)	0.0534471881366
  (0, 24862)	0.000612590176758
  (0, 24863)	0.000749593528972
  (0, 2

In [77]:
from scipy.stats.stats import pearsonr

###### Уберем повторяющиеся штуки

In [156]:
u_row_id = uid_to_row[256]
u_row = uid_profile[u_row_id]
u_recs = u_row.dot(ftr_iid_m)

row_to_iid = {row_id: iid for iid, row_id in iid_to_row.items()}

u_recs_nor=u_recs.todense().tolist()

iid_to_remove=training_df[training_df.uid==256]['iid'].unique()
iid_to_remove

array([  25,  155,  594, 1202, 1211, 1273, 1424, 1717, 1760], dtype=int64)

In [159]:
new_rec={}
for i in u_recs_nor[0]:
    iid=row_to_iid.get(u_recs_nor[0].index(i))
    if iid not in iid_to_remove: 
        new_rec[iid]=i

len(new_rec)

10089

In [245]:
new=np.argsort(u_recs.data)[::-1]
new

array([10088,  7814, 10089, ...,     0,    59,    21], dtype=int64)

In [260]:
sort_indices = np.argsort(u_recs.data)[::-1]
#for arg_id in np.argsort(u_recs.data)[:10:-1]:
for arg_id in sort_indices[:10]:
    print (arg_id, u_recs.data[arg_id])

10088 0.753456998087
7814 0.733437858658
10089 0.701529153184
9086 0.696706051422
6902 0.687498927811
9607 0.666887919766
7840 0.665352334314
8986 0.663852227725
10028 0.663492390483
7463 0.660308133017
8922 0.65232926508
7597 0.599205354583
6238 0.590783904771
8892 0.585824403898
10020 0.578219834812
8701 0.566251405474
7514 0.56523984421
6743 0.564624507434
8184 0.559967313021
1289 0.554722046698
7507 0.554674129779
6424 0.55393734717
8828 0.551984696344
7097 0.551263590958
8797 0.550793065032
8913 0.550303598551
6608 0.548966920948
9767 0.547707709206
3120 0.547269892997
7754 0.547211053825
5678 0.544889493276
8397 0.543277200056
6051 0.542853764405
8489 0.54209565799
9655 0.54043431282
5411 0.539841036144
9424 0.538299843105
8974 0.537584088478
8815 0.535983981986
9234 0.535694436001
9623 0.53503746285
9162 0.533428609616
7135 0.533015186607
3593 0.531405435574
8886 0.529020158166
8711 0.527830100229
6227 0.526935307058
5830 0.526460628173
6456 0.525718187171
9532 0.525370991191
57

6146 0.387434429207
7086 0.387424134332
6853 0.387239959442
3590 0.387172820067
4092 0.387128458355
7324 0.387092430493
4540 0.386989095275
7595 0.386883461674
9138 0.386870467359
8695 0.386866305548
1462 0.386785335967
7643 0.3867246311
9009 0.386711046453
9031 0.386601111017
7200 0.386587325435
4654 0.386515281091
3766 0.386460457758
7230 0.386410149065
7322 0.386322777435
5958 0.386285144821
7764 0.386194180006
9224 0.386183397851
9364 0.38616631194
6523 0.386113621023
5234 0.386084342604
9604 0.386083357265
4771 0.386082872116
8822 0.385999789156
9792 0.385927830696
9841 0.385863088944
8026 0.385843560457
7271 0.385675827329
9092 0.38564507224
4012 0.385590416761
4936 0.385564359345
9010 0.385339811264
240 0.385312764648
4664 0.385295575318
7396 0.385217200692
6044 0.385131517625
9755 0.385130552456
9294 0.385111138737
5855 0.385031292894
5763 0.385018951629
8943 0.385014464444
6848 0.384942149143
9122 0.38494047371
6708 0.384901617372
6425 0.384871664231
2364 0.384863891232
8724 0

6642 0.355767348583
7635 0.355701085486
7056 0.355534976207
6645 0.355534582833
5642 0.35550227283
9315 0.35544385588
5858 0.355437442784
2923 0.355436884188
4921 0.355436096925
4132 0.355379391998
7494 0.355364530684
1010 0.355344667111
7454 0.355313190263
2169 0.355309544879
5539 0.355252346365
180 0.355236100388
4512 0.355225125484
765 0.355164738846
5657 0.35515797144
2951 0.355138809324
9513 0.355131419557
9949 0.355117562004
6433 0.355063082764
7890 0.355062036641
7524 0.355036899278
1656 0.354988777561
6856 0.354919206022
6061 0.354887789969
8593 0.354876223961
8465 0.35483794703
7250 0.354825650507
7787 0.354816716808
5165 0.354786764634
6086 0.354783642669
9448 0.354680483112
6808 0.354646172701
3234 0.354633440519
9429 0.354614639241
1083 0.35459180908
9795 0.354506777586
8912 0.354423323654
7779 0.354377103313
8408 0.354321650955
6604 0.354251593766
8290 0.354241200324
2057 0.354206360416
2285 0.354165679218
5256 0.354150527806
3614 0.354105467956
8894 0.354078464237
9557 0.

300 0.324381324689
968 0.324333808409
2041 0.324303660849
7807 0.324292602388
8078 0.324284150005
1640 0.324283893932
4717 0.324276872902
3793 0.324257989893
7429 0.324227049418
4491 0.324223483813
9329 0.324209705294
8737 0.324178837115
8791 0.324166302971
3039 0.324146526224
8728 0.324075189482
6366 0.324066701221
4779 0.324038193637
4684 0.324023562784
789 0.324014375413
8682 0.323998358971
7106 0.323985630845
8538 0.323970753502
6474 0.323956480467
9683 0.323946768053
3348 0.323934342104
5268 0.323902661371
1628 0.323818230775
3144 0.323733702571
2800 0.32371097112
5849 0.323670576838
6036 0.323668812119
5931 0.323567394983
1498 0.323524272186
6409 0.323466399353
9371 0.323383635857
3642 0.323360061898
4740 0.323343725481
5193 0.323306227941
9576 0.32326432418
3639 0.323228581373
5039 0.323228532978
7173 0.323212820435
7576 0.323181776522
1168 0.323142138642
3526 0.323140303133
7402 0.323135351178
3852 0.32313130755
662 0.323111736515
5775 0.323084198374
1679 0.323056609663
7927 0.

3964 0.302582591054
2111 0.302568789748
9925 0.302560101162
5073 0.302550331867
7372 0.302542138112
3121 0.302518082871
8360 0.302477814761
5478 0.302422504739
10059 0.302408314569
6619 0.302389863012
1018 0.302386169747
7434 0.302373608523
3654 0.302349187993
2929 0.302309121326
9107 0.302219906531
6859 0.302174026818
3771 0.302147775281
4712 0.302122539112
1795 0.30208100525
3581 0.302079582844
3816 0.302078296199
8751 0.302052747175
4041 0.302029144126
4843 0.302006842387
7573 0.301880820458
4551 0.301848963869
1571 0.301846128908
9070 0.301822078094
1401 0.301802568625
1746 0.301775759509
3977 0.301769576444
7685 0.301753369673
1177 0.301749035056
268 0.301721003408
8497 0.301694141229
1170 0.301677816577
1221 0.301572686642
6774 0.301566179916
5904 0.301511806753
3439 0.301492826969
7502 0.301471267843
899 0.301468735518
8551 0.301465633569
7224 0.301463017379
3847 0.3014420503
5271 0.301424151322
7139 0.301414757182
5753 0.301414345131
9511 0.301365648384
3629 0.301365402829
2476

...

3524 0.229801391171
2968 0.229716963119
2979 0.229696902921
5708 0.22960176863
6245 0.229583901064
7636 0.229579011633
297 0.229561027839
8542 0.229533748938
504 0.229514869395
853 0.229476557219
2505 0.229451793359
4555 0.229391297461
8269 0.229389676953
4493 0.229356487855
7093 0.229342713126
10092 0.229339192689
1714 0.229306410324
987 0.229276735118
4158 0.229269762452
7264 0.229265589568
472 0.229206320905
763 0.229161661338
2494 0.229119377637
636 0.229089829997
1013 0.229053272158
8353 0.229041753638
4377 0.229020366151
5114 0.229003072768
6566 0.22899457447
8037 0.228916906625
9002 0.228902268119
365 0.228899146144
663 0.228892434412
8105 0.228835635731
3861 0.228810384703
4282 0.22879895012
5939 0.228797364136
1606 0.228747208542
1810 0.228716859175
8818 0.228692883583
1291 0.228655894375
5434 0.228653142652
824 0.228647775841
7208 0.22864124447
4990 0.228589957673
2659 0.228584172181
2225 0.22855974515
2373 0.228509067141
1399 0.228495365789
2728 0.228473579599
5591 0.2283297

4118 0.209257130102
2699 0.20922438603
3345 0.209210065012
9378 0.209196248431
5075 0.209115685068
7544 0.209070821436
903 0.209047775444
7584 0.209027618293
760 0.209021329773
252 0.20901678694
66 0.20896288375
2348 0.208940280623
1260 0.208810816244
8378 0.208805700558
7450 0.208769533147
5622 0.20868687227
6914 0.208642867264
6805 0.208566251543
2607 0.208525203606
674 0.208510554233
5563 0.208456955276
1483 0.208453409016
1403 0.208350695569
7121 0.208338722572
1920 0.208313509109
4328 0.20829145187
5784 0.208279557649
985 0.208265768445
8219 0.208234664286
618 0.208222054137
1147 0.20822035396
1299 0.208171767507
5363 0.208127404555
3294 0.208117934076
992 0.208073088365
106 0.208063858199
7878 0.207974363687
7630 0.207912606434
5278 0.207895514924
3081 0.207888591491
2447 0.207869563642
6140 0.207849408583
1492 0.20784016523
5544 0.207836576851
311 0.207829589033
3833 0.20781127375
7119 0.207787025518
91 0.207751013866
7989 0.207723645241
4313 0.207711649298
1267 0.20770084873
21

3277 0.183368167584
7562 0.183257566847
5390 0.183240929878
4860 0.183211685805
4811 0.183211473854
391 0.183177311161
7918 0.18314967514
3420 0.183059157081
4198 0.182985428762
3316 0.182769864773
5842 0.182749537082
4352 0.182670394099
3290 0.182634896557
4707 0.18258523933
7084 0.182566456378
3241 0.182392379057
1447 0.182365302879
9272 0.182357547687
994 0.182336523166
4294 0.182306562696
6377 0.182273024033
8051 0.182247177409
2838 0.182228391333
174 0.182178877364
600 0.182167788634
3513 0.182109119334
9335 0.182095926828
1384 0.182081884876
3978 0.182080385657
2895 0.18196293393
624 0.181937708399
5375 0.18191711559
8431 0.181876864151
6470 0.181868629193
1811 0.181822069651
3082 0.181745541789
6629 0.181684517199
7467 0.18164676046
7586 0.181630700085
2724 0.181587323571
6478 0.181568800351
4214 0.181560664011
7854 0.181424303871
7564 0.181413819472
3901 0.18136128748
4287 0.181359104013
2596 0.181303037294
8054 0.181189950994
2075 0.181148275398
86 0.181140521845
1003 0.181119

1052 0.150601800989
565 0.150587301404
5064 0.150458001898
2286 0.150372551441
2171 0.150351040493
8147 0.150327631562
1529 0.150323326198
6926 0.15032242558
9169 0.150309025843
7130 0.150304932555
8669 0.150234322587
5019 0.150215511841
4992 0.150163331899
9110 0.149940644994
285 0.149898613334
1727 0.149839078006
1090 0.149817744749
2839 0.149670677531
9131 0.149634685573
1304 0.149515422656
5649 0.149496066785
222 0.149483255992
4367 0.149388855356
5332 0.149387799522
4251 0.149241032656
4694 0.149182388183
4644 0.149150500453
8992 0.149046841615
5276 0.149024287226
7796 0.14899006668
9273 0.148988607661
5120 0.148967588698
1605 0.148925767333
7090 0.14891662569
9745 0.148914463259
1934 0.1488670768
8876 0.148834042115
1808 0.148795581871
6956 0.14874759361
1474 0.148681484333
1091 0.148679728109
849 0.14860394523
3692 0.148578745283
6669 0.148554144382
1313 0.148515888333
340 0.148473071475
4284 0.148393437767
3211 0.148373776716
75 0.148365721261
2391 0.148291453589
1726 0.1482582

3785 0.112693902339
4787 0.112489439468
271 0.112431873823
3963 0.112415034391
1709 0.112367036385
3685 0.11233405064
1610 0.112244381487
3830 0.112230331749
1930 0.112085036107
5233 0.112074338803
2173 0.112055243683
2592 0.112025725269
838 0.11197645878
371 0.111967126339
959 0.111937009429
424 0.111936480616
667 0.111870274142
3209 0.111792453917
4689 0.111768647529
3490 0.111706161467
3761 0.111694570468
483 0.111648497377
2555 0.111595127446
3086 0.111555894169
347 0.111328535448
604 0.111315497014
1556 0.111314100631
2072 0.111304845394
714 0.111279702161
823 0.111269499642
4100 0.111220670521
1946 0.111167648316
8249 0.110986286341
1464 0.110898236638
2959 0.110886110121
8608 0.110883754665
5536 0.110740146335
1735 0.110653674789
616 0.110646963141
6885 0.110605735904
2191 0.110458238836
3727 0.110421957711
253 0.110421866107
8381 0.110410491441
1782 0.110276014846
1481 0.110271583033
5022 0.110270144713
1508 0.110236243376
1472 0.110232344569
845 0.110109664607
4536 0.110104806

In [72]:
def get_recs2(uid, top):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        u_row = uid_profile[u_row_id]
        
        # самописный cosine similarity
        u_recs = u_row.dot(ftr_iid_m)
        
        u_recs_nor=u_recs.todense().tolist()
        
        iid_to_remove=training_df[training_df.uid==uid]['iid'].unique()

        
        new_rec={}
        for i in u_recs_nor[0]:
            iid=row_to_iid.get(u_recs_nor[0].index(i))
            if iid not in iid_to_remove: 
                new_rec[iid]=i
    return recs


In [267]:
def get_recs(uid, top):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        u_row = uid_profile[u_row_id]
        
        # самописный cosine similarity
        u_recs = u_row.dot(ftr_iid_m)
        
        # Уберем тех. кого пользователь смотрел
        iid_to_remove=training_df[training_df.uid==uid]['iid'].unique()
        # и узнаем их количество
        cnt_tu_del=len(iid_to_remove)

        sort_indices = np.argsort(u_recs.data)[::-1]
        
        for arg_id in sort_indices[:top+cnt_tu_del]:
            row_id = u_recs.indices[arg_id]
            score = u_recs.data[arg_id]
            iid=row_to_iid[row_id]
            if iid not in iid_to_remove: 
                recs[iid] = score
            if len(recs)>=top:
                return recs
    return recs

In [268]:
get_recs(256, 10)

{153: 0.70152915318433806,
 294: 0.69670605142188413,
 1091: 0.66349239048258302,
 1544: 0.6638522277250275,
 1796: 0.59920535458325619,
 2092: 0.66030813301739955,
 2194: 0.65232926507966604,
 3433: 0.68749892781138422,
 5050: 0.59078390477074139,
 5158: 0.66688791976610751}

In [269]:
    def get_batch_recs( uids, top):
        return {uid: get_recs(uid, top) for uid in uids}

In [270]:
%%time
rec_dict=get_batch_recs(test_df.uid,10)

Wall time: 6min 38s


In [271]:
# тупо берем первые 10 по суммарным баллам за просмотр
hit_ratio(rec_dict,test_dict)

0.07688921496698459

## Эксперименты и выводы

####  0-0 Результат по умолчанию (все фичи, TFIDF юзера по SUM(РЕВЬЮ*RATING), No Feature Selection). Результат теста hit-ratio (HR): 0.049

####  0-1 Результат по умолчанию (все фичи, TFIDF юзера по SUM(РЕВЬЮ), No Feature Selection). Убрал перемножение на рейтинг. Результат теста hit-ratio (HR):  0.0491562729273661

####  0-2 Вариант Турала (все фичи,  uid_profile по его объектам из iid_profile SUM(РЕВЬЮ*RATING), No Feature Selection).  Результат теста hit-ratio (HR): 0.06544387380777696

####  0-3 Вариант Турала (все фичи, uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста hit-ratio (HR):  0.06206896551724138  - как и предполагалось ухудшило результат

####  0-4 Вариант Турала (все фичи, iid_profile SUM(РЕВЬЮ), uid_profile по его объектам из iid_profile SUM(РЕВЬЮ*RATING), No Feature Selection).  Результат теста hit-ratio (HR):  0.06544387380777696 - точно такой же как и у iid_profile * RATING, наверное веса оценок iid при косинусной мере, не столь важны а вот для матирцы uid они позволяют более точно определить предпочтения пользователя 

####  1-1 Вариант Турала ( TFIDF , uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста TFIDF max_features=100 000  hit-ratio (HR):  0.0632428466617755


####  1-1 Вариант Турала ( TFIDF , uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста 
TFIDF max_features=100 000   hit-ratio (HR):  0.0632428466617755   
50 0000  (HR): 0.06456346294937637  
25 000 (HR): 0.06471019809244315
10 000 (HR): 0.0632428466617755